In [47]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

In [49]:
#Read Training and Test Data and concatenate the two df and set index

In [50]:
df_train = pd.read_csv("aug_train.csv")
df_test = pd.read_csv("aug_test.csv")
df = pd.concat([df_test, df_train])
df = df.set_index('enrollee_id')
df.head()

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
enrollee_id,,,,,,,,,,,,,
32403,city_41,0.827,Male,Has relevent experience,Full time course,Graduate,STEM,9,<10,NaN,1,21,NaN
9858,city_103,0.920,Female,Has relevent experience,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98,NaN
31806,city_21,0.624,Male,No relevent experience,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15,NaN
27385,city_13,0.827,Male,Has relevent experience,no_enrollment,Masters,STEM,11,10/49,Pvt Ltd,1,39,NaN
27724,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,72,NaN


In [51]:
#Checking for null values

In [52]:
null_info = pd.DataFrame(df.isnull().sum()).T.rename(index={0:'Null values'})
null_info = null_info.append(pd.DataFrame(df.isnull().sum()/df.shape[0] * 100).T.rename(index={0:'% Null values'}))
null_info

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
Null values,0.0,0.0,5016.000000,0.0,417.000000,512.000000,3125.000000,70.000000,6560.000000,6774.000000,463.000000,0.0,2129.000000
% Null values,0.0,0.0,23.563677,0.0,1.958942,2.405224,14.680321,0.328839,30.816931,31.822239,2.175036,0.0,10.001409


In [27]:
cat_var = ['gender', 'enrolled_university', 'education_level', 'major_discipline', 'company_size', 'company_type']

In [53]:
#Create dummies for columns having Not-Null values

In [54]:
df1 = pd.get_dummies(df[['city', 'relevent_experience']])
df1 = pd.DataFrame(df1, index=df.index)
df = df.drop(['city', 'relevent_experience'], axis=1)
df = pd.concat([df, df1], axis=1)
df.head()

,city_development_index,gender,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,...,city_city_9,city_city_90,city_city_91,city_city_93,city_city_94,city_city_97,city_city_98,city_city_99,relevent_experience_Has relevent experience,relevent_experience_No relevent experience
enrollee_id,,,,,,,,,,,,,,,,,,,,,
32403,0.827,Male,Full time course,Graduate,STEM,9,<10,NaN,1,21,...,0,0,0,0,0,0,0,0,1,0
9858,0.920,Female,no_enrollment,Graduate,STEM,5,NaN,Pvt Ltd,1,98,...,0,0,0,0,0,0,0,0,1,0
31806,0.624,Male,no_enrollment,High School,NaN,<1,NaN,Pvt Ltd,never,15,...,0,0,0,0,0,0,0,0,0,1
27385,0.827,Male,no_enrollment,Masters,STEM,11,10/49,Pvt Ltd,1,39,...,0,0,0,0,0,0,0,0,1,0
27724,0.920,Male,no_enrollment,Graduate,STEM,>20,10000+,Pvt Ltd,>4,72,...,0,0,0,0,0,0,0,0,1,0


In [55]:
#Imputing 'experience' columns using KNNImputer

In [56]:
df['experience'] = df['experience'].replace(['<1'], 0)
df['experience'] = df['experience'].replace(['>20'], 21)
df1 = df
df1 = df.drop(cat_var + list(['last_new_job']), axis=1)
df1.head()

,city_development_index,experience,training_hours,target,city_city_1,city_city_10,city_city_100,city_city_101,city_city_102,city_city_103,...,city_city_9,city_city_90,city_city_91,city_city_93,city_city_94,city_city_97,city_city_98,city_city_99,relevent_experience_Has relevent experience,relevent_experience_No relevent experience
enrollee_id,,,,,,,,,,,,,,,,,,,,,
32403,0.827,9,21,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9858,0.920,5,98,NaN,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
31806,0.624,0,15,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
27385,0.827,11,39,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
27724,0.920,21,72,NaN,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [30]:
imputer = KNNImputer()
df1_imputed = imputer.fit_transform(df1)
df1_imputed = pd.DataFrame(df1_imputed, index=df1.index, columns=df1.columns)
df1_imputed.head()

,city_development_index,experience,training_hours,target,city_city_1,city_city_10,city_city_100,city_city_101,city_city_102,city_city_103,...,city_city_9,city_city_90,city_city_91,city_city_93,city_city_94,city_city_97,city_city_98,city_city_99,relevent_experience_Has relevent experience,relevent_experience_No relevent experience
enrollee_id,,,,,,,,,,,,,,,,,,,,,
32403,0.827,9.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9858,0.920,5.0,98.0,0.2,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
31806,0.624,0.0,15.0,0.6,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
27385,0.827,11.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
27724,0.920,21.0,72.0,0.2,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
#Binning of Experience variable

In [31]:
bins = np.linspace(0, 25, 6)
labels = ['exp_one', 'exp_two', 'exp_three', 'exp_four', 'exp_five']
df1_imputed['exp_bins'] = pd.cut(df1_imputed['experience'], bins=bins, labels=labels)
df2 = pd.get_dummies(df1_imputed['exp_bins'])
df = df.drop(['experience'], axis=1)
df = pd.concat([df, df2], axis=1)
df.head()    

,city_development_index,gender,enrolled_university,education_level,major_discipline,company_size,company_type,last_new_job,training_hours,target,...,city_city_97,city_city_98,city_city_99,relevent_experience_Has relevent experience,relevent_experience_No relevent experience,exp_one,exp_two,exp_three,exp_four,exp_five
enrollee_id,,,,,,,,,,,,,,,,,,,,,
32403,0.827,Male,Full time course,Graduate,STEM,<10,NaN,1,21,NaN,...,0,0,0,1,0,0,1,0,0,0
9858,0.920,Female,no_enrollment,Graduate,STEM,NaN,Pvt Ltd,1,98,NaN,...,0,0,0,1,0,1,0,0,0,0
31806,0.624,Male,no_enrollment,High School,NaN,NaN,Pvt Ltd,never,15,NaN,...,0,0,0,0,1,0,0,0,0,0
27385,0.827,Male,no_enrollment,Masters,STEM,10/49,Pvt Ltd,1,39,NaN,...,0,0,0,1,0,0,0,1,0,0
27724,0.920,Male,no_enrollment,Graduate,STEM,10000+,Pvt Ltd,>4,72,NaN,...,0,0,0,1,0,0,0,0,0,1


In [ ]:
#Imputing 'last_new_job' columns using KNNImputer

In [32]:
df['last_new_job'] = df['last_new_job'].replace(['never'], 0)
df['last_new_job'] = df['last_new_job'].replace(['>4'], 5)
df1 = df
df1 = df.drop(cat_var, axis=1)
df1.head()    

,city_development_index,last_new_job,training_hours,target,city_city_1,city_city_10,city_city_100,city_city_101,city_city_102,city_city_103,...,city_city_97,city_city_98,city_city_99,relevent_experience_Has relevent experience,relevent_experience_No relevent experience,exp_one,exp_two,exp_three,exp_four,exp_five
enrollee_id,,,,,,,,,,,,,,,,,,,,,
32403,0.827,1,21,NaN,0,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
9858,0.920,1,98,NaN,0,0,0,0,0,1,...,0,0,0,1,0,1,0,0,0,0
31806,0.624,0,15,NaN,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
27385,0.827,1,39,NaN,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
27724,0.920,5,72,NaN,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1


In [33]:
imputer = KNNImputer()
df1_imputed = imputer.fit_transform(df1)
df1_imputed = pd.DataFrame(df1_imputed, index=df1.index, columns=df1.columns)
df1_imputed.head()

,city_development_index,last_new_job,training_hours,target,city_city_1,city_city_10,city_city_100,city_city_101,city_city_102,city_city_103,...,city_city_97,city_city_98,city_city_99,relevent_experience_Has relevent experience,relevent_experience_No relevent experience,exp_one,exp_two,exp_three,exp_four,exp_five
enrollee_id,,,,,,,,,,,,,,,,,,,,,
32403,0.827,1.0,21.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
9858,0.920,1.0,98.0,0.4,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
31806,0.624,0.0,15.0,0.8,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
27385,0.827,1.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
27724,0.920,5.0,72.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [34]:
bins = np.linspace(-1, 5, 7)
labels = ['lnj_zero', 'lnj_one', 'lnj_two', 'lnj_three', 'lnj_four', 'lnj_five']
df1_imputed['lnj_bins'] = pd.cut(df1_imputed['last_new_job'], bins=bins, labels=labels)
df2 = pd.get_dummies(df1_imputed['lnj_bins'])
df = df.drop(['last_new_job'], axis=1)
df = pd.concat([df, df2], axis=1)
df.head()    

,city_development_index,gender,enrolled_university,education_level,major_discipline,company_size,company_type,training_hours,target,city_city_1,...,exp_two,exp_three,exp_four,exp_five,lnj_zero,lnj_one,lnj_two,lnj_three,lnj_four,lnj_five
enrollee_id,,,,,,,,,,,,,,,,,,,,,
32403,0.827,Male,Full time course,Graduate,STEM,<10,NaN,21,NaN,0,...,1,0,0,0,0,1,0,0,0,0
9858,0.920,Female,no_enrollment,Graduate,STEM,NaN,Pvt Ltd,98,NaN,0,...,0,0,0,0,0,1,0,0,0,0
31806,0.624,Male,no_enrollment,High School,NaN,NaN,Pvt Ltd,15,NaN,0,...,0,0,0,0,1,0,0,0,0,0
27385,0.827,Male,no_enrollment,Masters,STEM,10/49,Pvt Ltd,39,NaN,0,...,0,1,0,0,0,1,0,0,0,0
27724,0.920,Male,no_enrollment,Graduate,STEM,10000+,Pvt Ltd,72,NaN,0,...,0,0,0,1,0,0,0,0,0,1


In [35]:
def find_category_mappings(df, variable):
    return {k: i for i, k in enumerate(df[variable].dropna().unique(), 0)}

In [36]:
def integer_encode(df , variable, ordinal_mapping):
    df[variable] = df[variable].map(ordinal_mapping)

In [37]:
mm = MinMaxScaler()
mappin = dict()
def imputations(df1, cols):
    df = df1.copy()
    for variable in cols:
        mappings = find_category_mappings(df, variable)
        mappin[variable] = mappings

    for variable in cols:
        integer_encode(df, variable, mappin[variable])

    sca = mm.fit_transform(df)
    knn_imputer = KNNImputer()
    knn = knn_imputer.fit_transform(sca)
    df.iloc[:, :] = mm.inverse_transform(knn)
    for i in df.columns:
        df[i] = round(df[i]).astype('int')

    for i in cols:
        inv_map = {v: k for k, v in mappin[i].items()}
        df[i] = df[i].map(inv_map)
    return df

In [41]:
df['company_size'] = df['company_size'].replace(['10/49'], '10-49')

In [42]:
df_cat = df[cat_var]
df_cat.head()

,gender,enrolled_university,education_level,major_discipline,company_size,company_type
enrollee_id,,,,,,
32403,Male,Full time course,Graduate,STEM,<10,Other
9858,Female,no_enrollment,Graduate,STEM,100-500,Pvt Ltd
31806,Male,no_enrollment,High School,STEM,100-500,Pvt Ltd
27385,Male,no_enrollment,Masters,STEM,10-49,Pvt Ltd
27724,Male,no_enrollment,Graduate,STEM,10000+,Pvt Ltd


In [43]:
df_imputed = imputations(df_cat, cat_var)
df_imputed.head()

,gender,enrolled_university,education_level,major_discipline,company_size,company_type
enrollee_id,,,,,,
32403,Male,Full time course,Graduate,STEM,<10,Other
9858,Female,no_enrollment,Graduate,STEM,100-500,Pvt Ltd
31806,Male,no_enrollment,High School,STEM,100-500,Pvt Ltd
27385,Male,no_enrollment,Masters,STEM,10-49,Pvt Ltd
27724,Male,no_enrollment,Graduate,STEM,10000+,Pvt Ltd


In [44]:
df = df.drop(cat_var, axis=1)
df = pd.concat([df, df_imputed], axis=1)
df.head()    

,city_development_index,training_hours,target,city_city_1,city_city_10,city_city_100,city_city_101,city_city_102,city_city_103,city_city_104,...,lnj_two,lnj_three,lnj_four,lnj_five,gender,enrolled_university,education_level,major_discipline,company_size,company_type
enrollee_id,,,,,,,,,,,,,,,,,,,,,
32403,0.827,21,NaN,0,0,0,0,0,0,0,...,0,0,0,0,Male,Full time course,Graduate,STEM,<10,Other
9858,0.920,98,NaN,0,0,0,0,0,1,0,...,0,0,0,0,Female,no_enrollment,Graduate,STEM,100-500,Pvt Ltd
31806,0.624,15,NaN,0,0,0,0,0,0,0,...,0,0,0,0,Male,no_enrollment,High School,STEM,100-500,Pvt Ltd
27385,0.827,39,NaN,0,0,0,0,0,0,0,...,0,0,0,0,Male,no_enrollment,Masters,STEM,10-49,Pvt Ltd
27724,0.920,72,NaN,0,0,0,0,0,1,0,...,0,0,0,1,Male,no_enrollment,Graduate,STEM,10000+,Pvt Ltd


In [45]:
df1 = pd.get_dummies(df[cat_var])
df1 = pd.DataFrame(df1, index=df.index)
df = df.drop(cat_var, axis=1)
df = pd.concat([df, df1], axis=1)
df.head()    

,city_development_index,training_hours,target,city_city_1,city_city_10,city_city_100,city_city_101,city_city_102,city_city_103,city_city_104,...,company_size_50-99,company_size_500-999,company_size_5000-9999,company_size_<10,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd
enrollee_id,,,,,,,,,,,,,,,,,,,,,
32403,0.827,21,NaN,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
9858,0.920,98,NaN,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
31806,0.624,15,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
27385,0.827,39,NaN,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
27724,0.920,72,NaN,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [46]:
df_test1 = df[0:len(df_test)]
df_train1 = df[len(df_test):]
df_test1.to_csv("df_test.csv")
df_train1.to_csv("df_train.csv")    